In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
customers = pd.read_csv("C:\\Users\\Swapnil\\Downloads\\crisis_recovery_online_food_startup\\RPC_18_Datasets\\dim_customer.csv")
restaurants = pd.read_csv("C:\\Users\\Swapnil\\Downloads\\crisis_recovery_online_food_startup\\RPC_18_Datasets\\dim_restaurant.csv")

In [34]:
orders = pd.read_csv("C:\\Users\\Swapnil\\Downloads\\crisis_recovery_online_food_startup\\RPC_18_Datasets\\fact_orders.csv")

In [3]:
customers.head()

,customer_id,signup_date,city,acquisition_channel
0,CUST000007,21-03-2025,Pune,Organic
1,CUST000008,07-02-2025,Kolkata,Referral
2,CUST000009,25-02-2025,Delhi,Paid
3,CUST000010,28-01-2025,Hyderabad,Paid
4,CUST000011,18-01-2025,Kolkata,Organic


In [9]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107776 entries, 0 to 107775
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   customer_id          107776 non-null  object        
 1   signup_date          107776 non-null  datetime64[ns]
 2   city                 107776 non-null  object        
 3   acquisition_channel  107776 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.3+ MB


In [5]:
customers.isnull().sum()

customer_id            0
signup_date            0
city                   0
acquisition_channel    0
dtype: int64

In [8]:
customers['signup_date'] = pd.to_datetime(customers['signup_date'])


C:\Users\Swapnil\AppData\Local\Temp\ipykernel_12528\1440559876.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  customers['signup_date'] = pd.to_datetime(customers['signup_date'])


In [35]:
orders.head()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,N,N
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,Y,N
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,N,N
3,ORD202501000124,CUST191820,REST19745,DP13859,2025-01-01 12:00:00,260.81,0.00,32.75,293.56,N,N
4,ORD202501006518,CUST033760,REST12962,DP09615,2025-01-01 12:00:00,280.33,0.00,25.57,305.90,N,N


In [36]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149166 entries, 0 to 149165
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             149166 non-null  object 
 1   customer_id          149166 non-null  object 
 2   restaurant_id        149166 non-null  object 
 3   delivery_partner_id  143531 non-null  object 
 4   order_timestamp      149166 non-null  object 
 5   subtotal_amount      149166 non-null  float64
 6   discount_amount      149166 non-null  float64
 7   delivery_fee         149166 non-null  float64
 8   total_amount         149166 non-null  float64
 9   is_cod               149166 non-null  object 
 10  is_cancelled         149166 non-null  object 
dtypes: float64(4), object(7)
memory usage: 12.5+ MB


In [37]:
orders['order_timestamp'] = pd.to_datetime(orders['order_timestamp'])

In [38]:
orders.isnull().sum()

order_id                  0
customer_id               0
restaurant_id             0
delivery_partner_id    5635
order_timestamp           0
subtotal_amount           0
discount_amount           0
delivery_fee              0
total_amount              0
is_cod                    0
is_cancelled              0
dtype: int64

In [39]:
orders.delivery_partner_id.describe()

count      143531
unique      15000
top       DP12792
freq           26
Name: delivery_partner_id, dtype: object

In [40]:
orders['delivery_partner_id'].value_counts()

delivery_partner_id
DP12792    26
DP11897    25
DP06208    23
DP09425    22
DP02195    22
           ..
DP01462     1
DP01199     1
DP01107     1
DP07749     1
DP07789     1
Name: count, Length: 15000, dtype: int64

Why delivery_partner_id is missing?
- 1. Order type or fulfillment mode
-    -- self delivery restaurants may not use third party delivery partners.
-    --  pickup orders typically don't involve delivery partners.
- 2. Operational Gaps
-    -- Partner assignment failed due to: System errors, partner availablity, manual override or fallback delivery
- 3. Data entry or integration issues
-   -- Missing values from: API integration errors, Delayed partner assignment, incomplete logs from external platforms
- 4. Orders cancelled before dispatch may never get a partner assigned

If missingness is valid(e.g pickup orders) then leave NULL

In [41]:
## check missing delivery_partner_id to understand  whether missingness is structural(pickup orders) or random
missing_summary = orders['delivery_partner_id'].isnull().groupby(orders['is_cancelled']).sum()
print(missing_summary)

is_cancelled
N       0
Y    5635
Name: delivery_partner_id, dtype: int64


All cancelled orders have delivery_partner_id = NULL, it likely means:
- Partner assignment happens after order confirmation. If an order is cancelled early (e.g. by customer or restaurant), it never reaches the dispatch stage.
- System design avoids assigning partners to orders that won’t be fulfilled. This prevents unnecessary routing or cost.
- Cancellation triggers before logistics engagement. Especially common in food delivery platforms where cancellations occur within seconds or minutes.

We should not impute delivery_partner_id for cancelled orders. Instead:
- Leave them as NULL — this reflects real operational flow.
- Flag them separately in your analysis to avoid skewing delivery metrics.
- Exclude them from SLA or delivery time modeling, since they were never dispatched.



In [42]:
orders[orders['delivery_partner_id'].isnull() & (orders['is_cancelled'] == 'Y')]

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
15,ORD202501011873,CUST050526,REST19508,NaN,2025-01-01 12:10:00,0.0,0.0,0.0,0.0,N,Y
24,ORD202501022002,CUST114265,REST08397,NaN,2025-01-01 12:15:00,0.0,0.0,0.0,0.0,N,Y
50,ORD202501012168,CUST119646,REST12616,NaN,2025-01-01 12:29:00,0.0,0.0,0.0,0.0,N,Y
54,ORD202501023485,CUST056936,REST03176,NaN,2025-01-01 12:32:00,0.0,0.0,0.0,0.0,N,Y
57,ORD202501000523,CUST195834,REST10055,NaN,2025-01-01 12:33:00,0.0,0.0,0.0,0.0,N,Y
...,...,...,...,...,...,...,...,...,...,...,...
149097,ORD202509001260,CUST150817,REST07214,NaN,2025-09-30 20:56:00,0.0,0.0,0.0,0.0,N,Y
149110,ORD202509009077,CUST024731,REST10666,NaN,2025-09-30 21:11:00,0.0,0.0,0.0,0.0,N,Y
149146,ORD202509005099,CUST040380,REST14432,NaN,2025-09-30 22:19:00,0.0,0.0,0.0,0.0,Y,Y
149156,ORD202509003738,CUST147227,REST05675,NaN,2025-09-30 22:38:00,0.0,0.0,0.0,0.0,Y,Y


In [43]:
## no imputation needed for delivery_partner_id in cancelled orders
## leave them as NULL to reflect real operational flow

In [44]:
orders.head()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,N,N
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,Y,N
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,N,N
3,ORD202501000124,CUST191820,REST19745,DP13859,2025-01-01 12:00:00,260.81,0.00,32.75,293.56,N,N
4,ORD202501006518,CUST033760,REST12962,DP09615,2025-01-01 12:00:00,280.33,0.00,25.57,305.90,N,N


In [45]:
restaurants.head()

,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active
0,REST12962,Flavours of Sweets Palace,Bengaluru,Chinese,Restaurant,26-40,N
1,REST14069,Royal Biryani Darbar,Ahmedabad,Fast Food,Restaurant,16-25,Y
2,REST08622,Spicy Wraps Point,Mumbai,North Indian,Restaurant,16-25,Y
3,REST19745,Delhi Express Palace,Ahmedabad,Pizza,Cloud Kitchen,16-25,Y
4,REST02383,Namma Mess Delight,Mumbai,Healthy,Restaurant,26-40,Y


In [46]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19995 entries, 0 to 19994
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   restaurant_id      19995 non-null  object
 1   restaurant_name    19995 non-null  object
 2   city               19995 non-null  object
 3   cuisine_type       19995 non-null  object
 4   partner_type       19995 non-null  object
 5   avg_prep_time_min  19995 non-null  object
 6   is_active          19995 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [47]:
restaurants.describe()

,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active
count,19995,19995,19995,19995,19995,19995,19995
unique,19995,4779,8,8,2,4,2
top,REST12962,Punjabi Express Central,Bengaluru,North Indian,Restaurant,16-25,Y
freq,1,12,4963,3942,14028,7987,18050


In [50]:
## lets create new column which indicates orders before crisis(Jan-May 2025) and during crisis(June-Sep 2025)
orders['period'] = np.where(
    (orders['order_timestamp'] >= pd.to_datetime('2025-06-01')) & (orders['order_timestamp'] <= pd.to_datetime('2025-09-30')),
    'Crisis',
    np.where(
        (orders['order_timestamp'] >= pd.to_datetime('2025-01-01')) & (orders['order_timestamp'] <= pd.to_datetime('2025-05-31')),
        'Pre-Crisis',
        None
    )
)

In [51]:
orders.head()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled,period
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,N,N,Pre-Crisis
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,Y,N,Pre-Crisis
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,N,N,Pre-Crisis
3,ORD202501000124,CUST191820,REST19745,DP13859,2025-01-01 12:00:00,260.81,0.00,32.75,293.56,N,N,Pre-Crisis
4,ORD202501006518,CUST033760,REST12962,DP09615,2025-01-01 12:00:00,280.33,0.00,25.57,305.90,N,N,Pre-Crisis


In [52]:
## save cleaned orders data to new csv
orders.to_csv("C:\\Users\\Swapnil\\Downloads\\crisis_recovery_online_food_startup\\RPC_18_Datasets\\fact_orders_cleaned.csv", index=False)